In [2]:
import pandas as pd
import string
import numpy as np

data = pd.read_excel(r'C:\Users\Francesc\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx', header=0, skiprows=1)

#TIPO DE FACTURA
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Punto de Venta'] + '-' + data['Número Desde']

# PROVEEDOR
data['Denominación Emisor'] = data['Denominación Emisor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Proveedor'] = data['Denominación Emisor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Emisor'].astype(str)

# TIPO DE CAMBIO
columnas = ['Imp. Neto Gravado', 'Imp. Neto No Gravado', 
            'Imp. Op. Exentas', 'Otros Tributos', 'IVA']
for col in columnas:
    data[col] = data[col] * data['Tipo Cambio']

# IVA (10.5 O 21)
data['% IVA'] = (data['IVA'] / data['Imp. Neto Gravado']) * 100
data['% IVA'] = np.where(abs(data['% IVA'] - 21) < 0.01, 21, data['% IVA'])
data['% IVA'] = np.where(abs(data['% IVA'] - 10.5) < 0.01, 10.5, data['% IVA'])

# CREAR COLUMNAS DE VALORES
data['NETO 10.5'] = np.nan
data['IVA 10.5'] = np.nan
data['NETO 21'] = np.nan
data['IVA 21'] = np.nan
data['NO GRAVADO'] = np.nan

# PONER 0 A TODAS LAS COLUMNAS
columnas = ['NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO']
for col in columnas:
    data[col] = data[col].fillna(0)

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)


# Asignamos los valores correspondientes a las nuevas columnas
data.loc[data['% IVA'] == 10.5, 'NETO 10.5'] = data['Imp. Neto Gravado'].astype(str)
data.loc[data['% IVA'] == 10.5, 'IVA 10.5'] = data['IVA'].astype(str)
data.loc[data['% IVA'] == 21, 'NETO 21'] = data['Imp. Neto Gravado'].astype(str)
data.loc[data['% IVA'] == 21, 'IVA 21'] = data['IVA'].astype(str)
data[col] = data[col].fillna(0)
# NO GRAVADO
data['NO GRAVADO'] = (data['Imp. Neto No Gravado'] + data['Imp. Op. Exentas'] + data['Otros Tributos'])

# PONER EN NEGATICO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    data.loc[data['Tipo2'] == 'Crédito', col] = -data[col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','Factura',
        'Proveedor','CUIT',
        'NETO 10.5', 'IVA 10.5',
        'NETO 21', 'IVA 21', 'NO GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,Factura,Proveedor,CUIT,NETO 10.5,IVA 10.5,NETO 21,IVA 21,NO GRAVADO
0,04/07/2024,00007-00279991,SUDAMERIA INCOMING SOLUTIONS SRL,33708398999,0.0,0.0,0.0,0.0,207853.44
1,04/07/2024,00007-00280255,SUDAMERIA INCOMING SOLUTIONS SRL,33708398999,0.0,0.0,0.0,0.0,499515.85000000003
2,04/07/2024,00008-00009592,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,285620.7
3,04/07/2024,00008-00009640,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,876677.8
4,04/07/2024,00007-00280261,SUDAMERIA INCOMING SOLUTIONS SRL,33708398999,0.0,0.0,0.0,0.0,506517.10000000003
5,04/07/2024,00008-00009608,CARNIVAL SRL,30676579938,27549.52,2892.7,0.0,0.0,0.78
6,04/07/2024,00008-00009609,CARNIVAL SRL,30676579938,27549.52,2892.7,0.0,0.0,0.78
7,04/07/2024,00007-00280267,SUDAMERIA INCOMING SOLUTIONS SRL,33708398999,0.0,0.0,0.0,0.0,1899859.2
8,04/07/2024,00007-00279961,SUDAMERIA INCOMING SOLUTIONS SRL,33708398999,0.0,0.0,0.0,0.0,69861.94
9,04/07/2024,00007-00056578,SUDAMERIA INCOMING SOLUTIONS SRL,33708398999,-0.0,-0.0,-0.0,-0.0,-2077377.4000000001
